In [57]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import datetime as dt
import itertools

In [58]:
data = pd.read_csv('movie_bd_v5.csv')

In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [59]:
data = pd.read_csv('movie_bd_v5.csv')

top = 5      # количество выводимых строк датасета
answers = {} # создадим словарь для ответов

# Разделяем значения в колонках на списки
data['cast_new'] = data.cast.apply(lambda x: x.split('|'))
data['director_new'] = data.director.apply(lambda x: x.split('|'))
data['genres_new'] = data.genres.apply(lambda x: x.split('|'))
data['production_companies'] = data.production_companies.apply(lambda x: x.split('|'))

# Задаем новую колонку: прибыль
data['profit'] = data.revenue - data.budget

# Приводим дату к стандартному формату
def date_split(x):
    my_date = x.split('/')
    y = dt.date(int(my_date[2]), int(my_date[0]), int (my_date[1]))
    return y 

data['release_date_new'] = data.release_date.apply(date_split)
data['month'] = data.release_date_new.apply(lambda x: x.month)

# Вводим колонку сезон
def def_season(month):
    if month in [1,2,12]: return 'winter'
    if month in [3,4,5]: return 'spring'
    if month in [6,7,8]: return 'summer'
    return 'outumn'

data['seasons'] = data.month.apply(def_season)


# Функция для создания датафрейма из списков значений, в привязке к ID фильмов.
# Например, все фильмы по режисерам, актерам, жанрам и т.д.
def make_df (source_df, key_column):
    
    # Создаем пустой словарь:
    # ключами словаря будут поля, которые необходимо связать с фильмами (жанры, актеры и т.д.)
    # значениями словаря будут списки ID соответсвующих фильмов
    movie_dict = {}

    # Перебираем каждую строку исходного датасета, внутри каждой строки перебираем каждый элемент списка.
    # Если элемент новый - заводим в словарь, если уже встречался - увеличиваем количество элементов в списке значений
    for i in source_df[key_column].index:
        for item in source_df[key_column][i]:
            if item in movie_dict.keys():
                movie_dict[item].append(source_df.imdb_id[i])
            else:
                movie_dict[item] = [source_df.imdb_id[i]]

    result_df = pd.DataFrame(movie_dict.items(), index = movie_dict.keys(), columns = [key_column,'movie_id'])
    result_df.drop([key_column], axis='columns', inplace=True)
    result_df['quantity'] = result_df.movie_id.apply(len)

    return result_df
data[data['release_year'] == 2014]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,cast_new,director_new,genres_new,profit,release_date_new,month,seasons
118,tt0816692,165000000,621752480,Interstellar,Matthew McConaughey|Jessica Chastain|Anne Hath...,Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,...,11/5/2014,8.0,2014,"[Matthew McConaughey, Jessica Chastain, Anne H...",[Christopher Nolan],"[Adventure, Drama, Science Fiction]",456752480,2014-11-05,11,outumn
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,...,7/30/2014,7.9,2014,"[Chris Pratt, Zoe Saldana, Dave Bautista, Vin ...",[James Gunn],"[Action, Science Fiction, Adventure]",603312399,2014-07-30,7,summer
120,tt1843866,170000000,714766572,Captain America: The Winter Soldier,Chris Evans|Scarlett Johansson|Sebastian Stan|...,Joe Russo|Anthony Russo,In heroes we trust.,After the cataclysmic events in New York with ...,136,Action|Adventure|Science Fiction,...,3/20/2014,7.6,2014,"[Chris Evans, Scarlett Johansson, Sebastian St...","[Joe Russo, Anthony Russo]","[Action, Adventure, Science Fiction]",544766572,2014-03-20,3,spring
121,tt2911666,20000000,78739897,John Wick,Keanu Reeves|Michael Nyqvist|Alfie Allen|Wille...,Chad Stahelski|David Leitch,Don't set him off.,"After the sudden death of his beloved wife, Jo...",101,Action|Thriller,...,10/22/2014,7.0,2014,"[Keanu Reeves, Michael Nyqvist, Alfie Allen, W...","[Chad Stahelski, David Leitch]","[Action, Thriller]",58739897,2014-10-22,10,outumn
122,tt1951265,125000000,752100229,The Hunger Games: Mockingjay - Part 1,Jennifer Lawrence|Josh Hutcherson|Liam Hemswor...,Francis Lawrence,Fire burns brighter in the darkness,Katniss Everdeen reluctantly becomes the symbo...,123,Science Fiction|Adventure|Thriller,...,11/18/2014,6.6,2014,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],"[Science Fiction, Adventure, Thriller]",627100229,2014-11-18,11,outumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,tt2183034,13000000,45300000,Earth to Echo,Teo Halm|Astro|Resse Hartwig|Ella Wahlestedt|J...,Dave Green,No one will ever believe our story.,After a construction project begins digging in...,89,Family|Adventure|Science Fiction,...,7/2/2014,5.7,2014,"[Teo Halm, Astro, Resse Hartwig, Ella Wahleste...",[Dave Green],"[Family, Adventure, Science Fiction]",32300000,2014-07-02,7,summer
235,tt2039393,25000000,39171130,The Gambler,Mark Wahlberg|John Goodman|Brie Larson|Michael...,Rupert Wyatt,The only way out is all in,Literature professor and gambler Jim Bennett's...,111,Thriller|Crime|Drama,...,12/25/2014,5.6,2014,"[Mark Wahlberg, John Goodman, Brie Larson, Mic...",[Rupert Wyatt],"[Thriller, Crime, Drama]",14171130,2014-12-25,12,winter
236,tt2319580,12700000,3430018,The Grand Seduction,Brendan Gleeson|Taylor Kitsch|Gordon Pinsent|L...,Don McKellar,The small town of Tickle Cove needed a doctor....,A small fishing village must procure a local d...,112,Comedy,...,5/30/2014,6.7,2014,"[Brendan Gleeson, Taylor Kitsch, Gordon Pinsen...",[Don McKellar],[Comedy],-9269982,2014-05-30,5,spring
237,tt2528814,17000000,29789000,God's Not Dead,Kevin Sorbo|Shane Harper|David A.R. White|Dean...,Harold Cronk,What do you believe?,College philosophy professor Mr. Radisson's cu...,113,Drama,...,3/21/2014,6.1,2014,"[Kevin Sorbo, Shane Harper, David A.R. White, ...",[Harold Cronk],[Drama],12789000,2014-03-21,3,spring


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [144]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)' #+
# если ответили верно, можете добавить комментарий со значком "+"

In [145]:
data[data.budget == max(data.budget)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,cast_new,director_new,genres_new,profit,release_date_new,month,seasons
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,...,5/11/2011,6.3,2011,"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...",[Rob Marshall],"[Adventure, Action, Fantasy]",641683000,2011-05-11,5,spring


# 2. Какой из фильмов самый длительный (в минутах)?

In [147]:
answers['2'] = 'Gods and Generals (tt0279111)' #+

In [35]:
data[data.runtime == max(data.runtime)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,director_new,profit,release_date_new,month,seasons
1157,tt0279111,56000000,12923936,Gods and Generals,"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke...",Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,"[Drama, History, War]","[Turner Pictures, Antietam Filmworks]",2/21/2003,5.8,2003,[Ronald F. Maxwell],-43076064,2003-02-21,2,winter


# 3. Какой из фильмов самый короткий (в минутах)?





In [148]:
answers['3']  = 'Winnie the Pooh (tt1449283)' #+

In [36]:
data[data.runtime == min(data.runtime)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,director_new,profit,release_date_new,month,seasons
768,tt1449283,30000000,14460000,Winnie the Pooh,"[Jim Cummings, Travis Oates, Jim Cummings, Bud...",Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,"[Animation, Family]","[Walt Disney Pictures, Walt Disney Animation S...",4/13/2011,6.8,2011,"[Stephen Anderson, Don Hall]",-15540000,2011-04-13,4,spring


# 4. Какова средняя длительность фильмов?


In [149]:
answers['4']  = 110 #+

In [37]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [150]:
answers['5']  = 107 #+

In [38]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [174]:
answers['6']  = 'Avatar (tt0499549)' #+

In [175]:
data[data.profit == max(data.profit)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,cast_new,director_new,genres_new,profit,release_date_new,month,seasons
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,...,12/10/2009,7.1,2009,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]",2544505847,2009-12-10,12,winter


# 7. Какой фильм самый убыточный? 

In [151]:
answers['7']  = 'The Lone Ranger (tt1210819)' #+

In [44]:
data[data.profit == min(data.profit)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,director_new,profit,release_date_new,month,seasons
1245,tt1210819,255000000,89289910,The Lone Ranger,"[Johnny Depp, Armie Hammer, William Fichtner, ...",Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"[Action, Adventure, Western]","[Walt Disney Pictures, Jerry Bruckheimer Films...",7/3/2013,6.0,2013,[Gore Verbinski],-165710090,2013-07-03,7,summer


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [152]:
answers['8']  = 1478 #+

In [47]:
len(data[data.profit > 0]['profit'])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [153]:
answers['9']  = 'The Dark Knight (tt0468569)' #+'

In [49]:
data_2008 = data[data.release_year == 2008]
data_2008[data_2008.profit == max(data_2008.profit)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,director_new,profit,release_date_new,month,seasons
599,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",7/16/2008,8.1,2008,[Christopher Nolan],816921825,2008-07-16,7,summer


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [154]:
answers['10']  = 'The Lone Ranger (tt1210819)' #+

In [51]:
data_2012_2014 = data[data.release_year.between(2012, 2014)]
data_2012_2014[data_2012_2014.profit == min(data_2012_2014.profit)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,director_new,profit,release_date_new,month,seasons
1245,tt1210819,255000000,89289910,The Lone Ranger,"[Johnny Depp, Armie Hammer, William Fichtner, ...",Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"[Action, Adventure, Western]","[Walt Disney Pictures, Jerry Bruckheimer Films...",7/3/2013,6.0,2013,[Gore Verbinski],-165710090,2013-07-03,7,summer


# 11. Какого жанра фильмов больше всего?

In [155]:
answers['11']  = 'Drama' #+

In [97]:
make_df(data, 'genres_new').sort_values('quantity', ascending = False).head(top)

,movie_id,quantity
Drama,"[tt1663202, tt3659388, tt0470752, tt3460252, t...",782
Comedy,"[tt2293640, tt2096673, tt2120120, tt1596363, t...",683
Thriller,"[tt0369610, tt1392190, tt2908446, tt2820852, t...",596
Action,"[tt0369610, tt1392190, tt2488496, tt2820852, t...",582
Adventure,"[tt0369610, tt1392190, tt2908446, tt2488496, t...",415


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [156]:
answers['12']  = 'Drama' #+

In [64]:
# Создаем пустой словарь для накопления количества жанров
genres_dict = {}

# Перебираем каждую строку датасета data.genres, внутри каждой строки перебираем каждый элемент списка.
# Если жанр новый - заводим в словарь, если уже встречался - увеличиваем счетчик конкретного жанра
# и накапливаем в значениях словаря (в спсике) бюджет, сборы, прибыль или убыток.
for i in data.genres.index:
    for genre in data['genres_new'][i]:
        if genre in genres_dict.keys():
            genres_dict[genre][0] += 1
            genres_dict[genre][1] += data['profit'][i]
            # Проверяем доходность и считаем количество прибыльных фильмов
            if data['profit'][i] > 0:
                genres_dict[genre][2] += 1
        else:
            if data['profit'][i] > 0:
                genres_dict[genre] = [1, data['profit'][i],1]
            else:
                genres_dict[genre] = [1, data['profit'][i],0]

genres_df = pd.DataFrame(genres_dict.items(), columns = ['genre','temp'])

# Разделяем список со значениями на отдельные колонки
genres_df['quantity'] = genres_df.temp.apply(lambda x: x.pop(0)) 
genres_df['profit'] = genres_df.temp.apply(lambda x: x.pop(0)) 
genres_df['sucses'] = genres_df.temp.apply(lambda x: x.pop(0)) 
genres_df.drop(['temp'], axis='columns', inplace=True)

g_filter = ['Drama', 'Comedy', 'Action', 'Thriller', 'Adventure']
genres_df[genres_df.genre.isin(g_filter)].sort_values('sucses',ascending = False)

,genre,quantity,profit,sucses
7,Drama,782,49257224159,560
10,Comedy,683,59774752721,551
3,Thriller,596,46740546578,446
0,Action,582,80187900065,444
1,Adventure,415,86070466981,337


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [157]:
answers['13']  = 'Peter Jackson' #+

In [98]:
data.groupby(['director']).sum().sort_values('revenue', ascending = False).head(top)

,budget,revenue,runtime,vote_average,release_year,profit,month
director,,,,,,,
Peter Jackson,1288000000,6490593685,1355,58.1,16059,5202593685,95
Christopher Nolan,1005000000,4167548502,1135,61.1,16057,3162548502,63
David Yates,775000000,4154295625,567,29.6,8037,3379295625,30
Michael Bay,1126000000,3886938960,1209,50.0,16063,2760938960,47
J.J. Abrams,740000000,3579169916,633,35.1,10054,2839169916,33


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [158]:
answers['14']  = 'Robert Rodriguez' #+/-

In [67]:
directors = data[data.genres.str.contains('Action')].groupby('director').count().sort_values('imdb_id',ascending = False).imdb_id
my_filter = ['Ridley Scott', 'Guy Ritchie', 'Robert Rodriguez', 'Quentin Tarantino', 'Tony Scott']
directors.loc[my_filter].sort_values(ascending = False)

director
Robert Rodriguez     6
Ridley Scott         6
Tony Scott           5
Guy Ritchie          4
Quentin Tarantino    3
Name: imdb_id, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [171]:
answers['15']  = 'Chris Hemsworth' #+

In [76]:
# формируем датасет из актеров в привязке к фильмам
cast_df = make_df(data,'cast_new')

# создаем пустой список для накопления сумм сборов
a=[]
data_2012 = data[data['release_year'] == 2012]

for s in range(len(cast_df)):
    a.append(data_2012[data_2012['imdb_id'].isin(cast_df['movie_id'][s])]['revenue'].sum())

# добавляем сборы в датасет с актерами 
cast_df['revenue'] = a

# накладываем фильтр по актерам из вопроса и выводим результат
cast_filter = ['Nicolas Cage','Tom Hardy','Chris Hemsworth','Jim Sturgess','Emma Stone']
cast_df[cast_df.index.isin(cast_filter)].sort_values('revenue', ascending = False)

,movie_id,quantity,revenue
Chris Hemsworth,"[tt2395427, tt2717822, tt1390411, tt1524930, t...",12,2027450773
Tom Hardy,"[tt1392190, tt1663202, tt3569230, tt1014763, t...",8,1134717867
Emma Stone,"[tt1243974, tt2562232, tt1872181, tt1156398, t...",13,752215857
Jim Sturgess,"[tt1023114, tt0478087, tt1563738, tt1371111, t...",6,138589343
Nicolas Cage,"[tt2467046, tt0448011, tt1095217, tt0375568, t...",27,2106557


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [172]:
answers['16']  = 'Matt Damon' #+

In [99]:
# формируем новый датасет из высокобюджетных фильмов (выше среднего бюджета)
high_budget = data[data['budget'] >= data['budget'].mean()]

# формируем дадасет по актерам и фильмам и выводим результат
cast_df = make_df(high_budget, 'cast_new')
cast_df.sort_values('quantity',ascending = False).head(top)

,movie_id,quantity
Matt Damon,"[tt3659388, tt2177771, tt1057500, tt0947810, t...",18
Adam Sandler,"[tt2120120, tt2510894, tt1201167, tt1375670, t...",17
Angelina Jolie,"[tt1587310, tt0944835, tt1243957, tt0146316, t...",16
Samuel L. Jackson,"[tt2802144, tt1843866, tt1386588, tt0489099, t...",15
Eddie Murphy,"[tt0892791, tt0126029, tt0240462, tt0765476, t...",15


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [170]:
answers['17']  = 'Action' #+

In [100]:
# Создаем отдельный датасет для Кейджа и считаем количество жанров
data_Cage = data[data['cast'].str.contains('Nicolas Cage')]
genres_df = make_df(data_Cage, 'genres_new')
genres_df.sort_values('quantity', ascending = False).head(top)

,movie_id,quantity
Action,"[tt2467046, tt0448011, tt0375568, tt1250777, t...",17
Thriller,"[tt2467046, tt0814022, tt1502404, tt1071875, t...",15
Drama,"[tt0448011, tt1095217, tt0963966, tt1674784, t...",12
Crime,"[tt1095217, tt1250777, tt0814022, tt1502404, t...",10
Fantasy,"[tt0963966, tt1502404, tt1071875, tt0479997, t...",8


# 18. Самый убыточный фильм от Paramount Pictures

In [165]:
answers['18']  = 'K-19: The Widowmaker (tt0267626)' #+

In [90]:
studios_df = make_df(data, 'production_companies')
m_filter = studios_df.loc['Paramount Pictures'].movie_id
data_st = data[(data['imdb_id'].isin(m_filter))]
data_st[data_st['profit'] == data_st.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,cast_new,director_new,genres_new,profit,release_date_new,month,seasons
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,...,7/19/2002,6.0,2002,"[Harrison Ford, Liam Neeson, Peter Sarsgaard, ...",[Kathryn Bigelow],"[Thriller, Drama, History]",-64831034,2002-07-19,7,summer


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [166]:
answers['19']  = 2015 #+ 

In [101]:
data.groupby('release_year').sum().sort_values('revenue',ascending = False).head(top)

,budget,revenue,runtime,vote_average,profit,month
release_year,,,,,,
2015,6780630004,25449202382,13355,755.2,18668572378,842
2014,7008050000,23405862953,13444,774.5,16397812953,854
2013,7970620000,23213799791,15039,846.9,15243179791,943
2012,7002000000,23079001687,12145,682.1,16077001687,724
2011,7946550531,22676791872,15355,868.5,14730241341,949


# 20. Какой самый прибыльный год для студии Warner Bros?

In [167]:
answers['20']  = 2014  #По моему, верный ответ - 2008 (см. решение). Не понимаю, почему валидатор считает верным 2014. 

In [117]:
y_filter = [2014, 2008, 2012, 2010, 2015]
new_data = data[data['release_year'].isin(y_filter)]

studios_df = make_df(new_data, 'production_companies')
m_filter = studios_df.loc['Warner Bros.'].movie_id
data_st = new_data[(new_data.imdb_id.isin(m_filter))]

data_st.groupby('release_year').sum().sort_values('profit',ascending = False)

,budget,revenue,runtime,vote_average,profit,month
release_year,,,,,,
2008,916000000,3050595031,1366,72.0,2134595031,81
2014,887600000,2773903827,1476,81.4,1886303827,87
2010,954000000,2807128844,1341,74.5,1853128844,80
2012,611500000,1869520056,579,30.0,1258020056,33
2015,820800003,1691168351,1358,75.9,870368348,82


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [168]:
answers['21']  = 'Сентябрь' #+

In [120]:
data.groupby('month').imdb_id.count().sort_values(ascending = False).head(top)

month
9     227
12    190
10    186
8     161
3     156
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [121]:
answers['22']  = 450 #+

In [122]:
seasons = data.groupby('month').imdb_id.count()
seasons[5:8].sum()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [123]:
answers['23']  = 'Peter Jackson' #-

In [144]:
director_df = make_df(data, 'director_new')

a = [] # вспомогательный список для накопления количества фильмов в разрезе режисеров и сезонов
b = [] # список для расчета продуктивности зимы: отношение количества фильмов,
       #выпущенных зимой конкретным режисером к общему количеству фильмов этого режисера

for director in range(len(director_df)):
    a.append(pd.Series(data[data['imdb_id'].isin(director_df['movie_id'][director])]['seasons']))
    if 'winter' in list(a[director]):
        b.append(a[director].value_counts(1).loc['winter'])
    else:
        b.append(0)

director_df['seasons'] = b
director_df.sort_values('seasons', ascending = False).head(10)
m_filter = ['Steven Soderbergh', 'Christopher Nolan', 'Clint Eastwood', 'Ridley Scott', 'Peter Jackson']
director_df[director_df.index.isin(m_filter)].sort_values('seasons', ascending = False)


,movie_id,quantity,seasons
Peter Jackson,"[tt2310332, tt0380510, tt0120737, tt0167261, t...",8,0.875000
Clint Eastwood,"[tt2179136, tt1057500, tt1212419, tt1205489, t...",12,0.500000
Steven Soderbergh,"[tt1130080, tt0240772, tt1598778, tt0307479, t...",13,0.461538
Ridley Scott,"[tt3659388, tt1528100, tt0955308, tt0265086, t...",12,0.250000
Christopher Nolan,"[tt0816692, tt1375666, tt0468569, tt0278504, t...",8,0.000000


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [128]:
answers['24']  = 'Four By Two Productions' #+

In [133]:
studios_df = make_df(data, 'production_companies')

# "a" и "b" - вспомогательные списки, итоговые средние длины названий в списке "с"
a = []
b = []
c = []
for s in range(len(studios_df)):
    a.append(pd.Series(data[data['imdb_id'].isin(studios_df['movie_id'][s])]['original_title']))
    b.append(''.join(list(a[s])))
    c.append(len(b[s])/len(a[s]))

studios_df['name_len'] = c
m_filter = ['Universal Pictures (Universal)', 'Warner Bros.', 'Jim Henson Company, The', 'Paramount Pictures' , 'Four By Two Productions']
studios_df[studios_df.index.isin(m_filter)].sort_values('name_len', ascending = False)


,movie_id,quantity,name_len
Four By Two Productions,[tt0443453],1,83.000000
"Jim Henson Company, The",[tt1698641],1,59.000000
Paramount Pictures,"[tt1340138, tt1596363, tt2381249, tt1528854, t...",122,17.032787
Warner Bros.,"[tt1617661, tt2126355, tt1638355, tt2361509, t...",168,15.571429


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [134]:
answers['25']  = 'Midnight Picture Show' #+

In [136]:
studios_df = make_df(data, 'production_companies')

a = []
b = []
for s in range(len(studios_df)):
    a.append(pd.Series(data[data['imdb_id'].isin(studios_df['movie_id'][s])]['overview']))
    # сначала соединяем описание всех фильмов по конкретной студии в одну строку, потом считаем количество слов 
    b.append(len(' '.join(list(a[s])).split(' '))/len(a[s]))    

studios_df['overview_len'] = b
m_filter = ['Universal Pictures', 'Warner Bros.', 'Midnight Picture Show', 'Paramount Pictures', 'Total Entertainment']
studios_df[studios_df.index.isin(m_filter)].sort_values('overview_len', ascending = False)


,movie_id,quantity,overview_len
Midnight Picture Show,[tt0805570],1,175.000000
Total Entertainment,[tt1324999],1,86.000000
Paramount Pictures,"[tt1340138, tt1596363, tt2381249, tt1528854, t...",122,55.278689
Warner Bros.,"[tt1617661, tt2126355, tt1638355, tt2361509, t...",168,54.601190
Universal Pictures,"[tt2820852, tt2293640, tt2637276, tt2848292, t...",173,53.682081


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [137]:
answers['26']  = 'Inside Out, The Dark Knight, 12 Years a Slave' #+

In [138]:
data[data['vote_average'] >= data['vote_average'].quantile(0.99)]
m_filter = ['Inside Out', 'The Dark Knight', '12 Years a Slave', 'BloodRayne', 'The Adventures of Rocky & Bullwinkle',
            'Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down', '300', 'Lucky Number Slevin',
            'Kill Bill: Vol. 1', 'Upside Down', 'Inside Out', 'Iron Man']

data[(data['vote_average'] >= data['vote_average'].quantile(0.99)) & data['original_title'].isin(m_filter)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,cast_new,director_new,genres_new,profit,release_date_new,month,seasons
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,...,6/9/2015,8.0,2015,"[Amy Poehler, Phyllis Smith, Richard Kind, Bil...",[Pete Docter],"[Comedy, Animation, Family]",678708609,2015-06-09,6,summer
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,7/16/2008,8.1,2008,"[Christian Bale, Michael Caine, Heath Ledger, ...",[Christopher Nolan],"[Drama, Action, Crime, Thriller]",816921825,2008-07-16,7,summer
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,Adventure|Fantasy|Action,...,12/1/2003,7.9,2003,"[Elijah Wood, Ian McKellen, Viggo Mortensen, L...",[Peter Jackson],"[Adventure, Fantasy, Action]",1024888979,2003-12-01,12,winter
1191,tt2024544,20000000,187000000,12 Years a Slave,Chiwetel Ejiofor|Michael Fassbender|Lupita Nyo...,Steve McQueen,The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,Drama|History,...,10/18/2013,7.9,2013,"[Chiwetel Ejiofor, Michael Fassbender, Lupita ...",[Steve McQueen],"[Drama, History]",167000000,2013-10-18,10,outumn


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [142]:
answers['27']  = 'Daniel Radcliffe & Rupert Grint' # +/- Не успевал к дедлайну, взял командное решение.
                                                   # Разобрался как работает. Хорошее решение

In [140]:
counter = Counter()
for casts in data['cast_new']:
    for comb in itertools.combinations(casts,2):
        counter[comb] +=1
max_value = sorted(counter.values(),reverse = True)[0]
for k,v in counter.items():
    if v == max_value:
        print(k,v)

('Daniel Radcliffe', 'Rupert Grint') 8
('Daniel Radcliffe', 'Emma Watson') 8


# Submission

In [177]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'21': 'Сентябрь',
 '22': 450,
 '23': 'Ridley Scott',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint',
 '1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '6': 'Avatar (tt0499549)'}

In [176]:
# и убедиться что ни чего не пропустил)
len(answers)

27